In [1]:
%matplotlib inline
import math
import pandas as pd
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.tsa.vector_ar.var_model as VAR
import PPCA_MR as p
import statsmodels.api as sm

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=2)
pylab.rcParams['figure.figsize'] = (24, 18)

# Hand cleansed
Hand cleaned to removed duplicates

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:158: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


# Amihud methodology

Amihud (2002) developed the illiquidity measure expressed by:

$$ Amihud =\frac{1}{N} \sum \frac{|r_t|}{\text{dvol}_t} $$

where $r_t$ is the return on day $t$ and $Volume_t$ is the dollar volume on day $t$. 

The benefit of taking the daily return is to weight the move by change of price and the volume.


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:42: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:46: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:49: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

# Academic papers
Based on Illin, A., Raiko, T. 2010 paper called "Practical Approaches to Principal Component Analysis in the Presence of Missing Values"

Tipping and Bishop (1999) introducted probablistic formulation of PCA (PPCA)


Use a flavor of Probabilistic PCA that is robust to missing data (see Ilin and Raiko 2010). Rather than the vanilla one-shot SVD, PPCA uses an iterative EM procedure/fixed point algorithm. From an initial guess, it’ll alternatively interpolate missing data and update the components until convergence.

On a side note, PPCA interpolates using information from all series, making it a multivariate interpolator. Be careful though, fitting the model to too many components will likely lead to overfitting problems in the interpolated data.

Also note that I am using the standardization that is mentioned in Korajecyk and Sadka (2008) of expanding zscores


In [5]:
#data=calc_zscore_ew(amihud.dropna(how='all')).dropna(how='all')
data=calc_zscore_expanding_window(amihud.dropna(how='all'),3).dropna(how='all')

tsmom_model.py:273: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=3,window=100000000,center=False).mean()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)
tsmom_model.py:273: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=3,window=100000000,center=False).std()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)


In [6]:
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

In [42]:
sector_PC={}
sector_variance_explained=pd.DataFrame()

no_pc=3
for sect in d_map.keys():
    try: 
        PPCA=p.PPCA()
        PPCA.fit(np.array(sector_zscores[sect]),d=no_pc)
        sector_variance_explained[sect] = PPCA.var_exp
        pcs=pd.DataFrame()
        for i in range(0,no_pc,1):
            x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
            x.set_axis(sector_zscores[sect].index)
            pcs[i]=x
        sector_PC[sect] = pcs
    except:
        print sect

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  del sys.path[0]


In [43]:
no_fx = [i for i in data.columns if i not in d_map['Currencies']]
PPCA=p.PPCA()
PPCA.fit(np.array(data[no_fx]),d=no_pc)
sector_variance_explained[sect] = PPCA.var_exp
pcs=pd.DataFrame()
for i in range(0,no_pc,1):
    x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
    x.set_axis(sector_zscores[sect].index)
    pcs[i]=x
sector_PC['Total'] = pcs

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  


In [44]:
prin_factors=pd.DataFrame()
for sector in  ['Agriculturals',
                 'Energies',
                 'Equities',
                 'Metals',
                 'Fixed Income',
                 'Total']:
    prin_factors['Amihud '+sector]=sector_PC[sector].mean(axis=1)

In [45]:
res = VAR.VAR(endog=prin_factors)

In [46]:
res.select_order(14)

                 VAR Order Selection                  
            aic          bic          fpe         hqic
------------------------------------------------------
0       -0.1426     -0.04190       0.8671      -0.1018
1       -8.320*      -7.615*   0.0002436*      -8.035*
2        -8.306       -6.997    0.0002474       -7.776
3        -8.293       -6.379    0.0002513       -7.518
4        -8.143       -5.626    0.0002931       -7.124
5        -8.065       -4.944    0.0003194       -6.801
6        -7.916       -4.190    0.0003753       -6.407
7        -7.750       -3.420    0.0004500       -5.997
8        -7.695       -2.760    0.0004862       -5.697
9        -7.635       -2.096    0.0005309       -5.393
10       -7.591       -1.448    0.0005752       -5.104
11       -7.455      -0.7076    0.0006891       -4.723
12       -7.314      0.03791    0.0008382       -4.337
13       -7.405       0.5507    0.0008173       -4.184
14       -7.334        1.226    0.0009498       -3.868
* Minimum


{'aic': 1, 'bic': 1, 'fpe': 1, 'hqic': 1}

In [47]:
x=res.fit()

In [57]:
test=x.test_causality('Amihud Total','Amihud Total')

Granger causality f-test
   Test statistic   Critical Value          p-value         df
--------------------------------------------------------------
       545.996723         3.849181            0.000  (1, 1206)
H_0: ['Amihud Total'] do not Granger-cause Amihud Total
Conclusion: reject H_0 at 5.00% significance level


In [59]:
test.values()[4]

6.5374832709925606e-100

In [60]:
pvalues=pd.DataFrame()
for col in x.tvalues.columns:
    s=pd.Series()
    for col2 in x.tvalues.columns:
        s[col2]=x.test_causality(col,col2).values()[4]
    pvalues[col]=s
    

Granger causality f-test
   Test statistic   Critical Value          p-value         df
--------------------------------------------------------------
       464.363009         3.849181            0.000  (1, 1206)
H_0: ['Amihud Agriculturals'] do not Granger-cause Amihud Agriculturals
Conclusion: reject H_0 at 5.00% significance level
Granger causality f-test
   Test statistic   Critical Value          p-value         df
--------------------------------------------------------------
         0.003255         3.849181            0.955  (1, 1206)
H_0: ['Amihud Energies'] do not Granger-cause Amihud Agriculturals
Conclusion: fail to reject H_0 at 5.00% significance level
Granger causality f-test
   Test statistic   Critical Value          p-value         df
--------------------------------------------------------------
         0.142471         3.849181            0.706  (1, 1206)
H_0: ['Amihud Equities'] do not Granger-cause Amihud Agriculturals
Conclusion: fail to reject H_0 at 5.00% si

In [80]:
print pvalues.round(3).T.to_latex()

\begin{tabular}{lrrrrrr}
\toprule
{} &  Amihud Agriculturals &  Amihud Energies &  Amihud Equities &  Amihud Metals &  Amihud Fixed Income &  Amihud Total \\
\midrule
Amihud Agriculturals &                 0.000 &            0.955 &            0.706 &          0.321 &                0.243 &         0.555 \\
Amihud Energies      &                 0.161 &            0.000 &            0.234 &          0.917 &                0.047 &         0.492 \\
Amihud Equities      &                 0.874 &            0.282 &            0.000 &          0.324 &                0.772 &         0.771 \\
Amihud Metals        &                 0.429 &            0.741 &            0.813 &          0.000 &                0.441 &         0.014 \\
Amihud Fixed Income  &                 0.684 &            0.730 &            0.081 &          0.309 &                0.000 &         0.852 \\
Amihud Total         &                 0.933 &            0.297 &            0.222 &          0.380 &                0.895 

In [85]:
x.tvalues.T.round(2)

,const,L1.Amihud Agriculturals,L1.Amihud Energies,L1.Amihud Equities,L1.Amihud Metals,L1.Amihud Fixed Income,L1.Amihud Total
Amihud Agriculturals,0.00,21.55,-0.06,0.38,0.99,-1.17,0.59
Amihud Energies,0.98,-1.40,13.34,1.19,0.10,1.99,0.69
Amihud Equities,2.02,0.16,1.08,30.77,0.99,-0.29,-0.29
Amihud Metals,-1.08,0.79,-0.33,0.24,13.28,-0.77,-2.47
Amihud Fixed Income,-2.73,-0.41,0.35,1.74,1.02,25.13,0.19
Amihud Total,0.99,0.08,1.04,-1.22,0.88,0.13,23.37


In [77]:
columns=[ u'L1.Amihud Agriculturals', u'L1.Amihud Energies',
       u'L1.Amihud Equities', u'L1.Amihud Metals', u'L1.Amihud Fixed Income',
       u'L1.Amihud Total']

In [78]:
print x.params.T[columns].round(3).to_latex()

\begin{tabular}{lrrrrrr}
\toprule
{} &  L1.Amihud Agriculturals &  L1.Amihud Energies &  L1.Amihud Equities &  L1.Amihud Metals &  L1.Amihud Fixed Income &  L1.Amihud Total \\
\midrule
Amihud Agriculturals &                    0.858 &              -0.005 &               0.016 &             0.066 &                  -0.062 &            0.019 \\
Amihud Energies      &                   -0.036 &               0.709 &               0.032 &             0.004 &                   0.067 &            0.014 \\
Amihud Equities      &                    0.004 &               0.063 &               0.908 &             0.046 &                  -0.011 &           -0.007 \\
Amihud Metals        &                    0.026 &              -0.023 &               0.008 &             0.736 &                  -0.034 &           -0.066 \\
Amihud Fixed Income  &                   -0.011 &               0.019 &               0.047 &             0.044 &                   0.868 &            0.004 \\
Amihud Total   

In [50]:
print res.fit(1).summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 22, May, 2018
Time:                     10:17:11
--------------------------------------------------------------------
No. of Equations:         6.00000    BIC:                   -7.64329
Nobs:                     208.000    HQIC:                  -8.04471
Log likelihood:          -863.845    FPE:                0.000244313
AIC:                     -8.31721    Det(Omega_mle):     0.000200307
--------------------------------------------------------------------
Results for equation Amihud Agriculturals
                             coefficient       std. error           t-stat            prob
------------------------------------------------------------------------------------------
const                           0.000069         0.105357            0.001           0.999
L1.Amihud Agriculturals         0.858325         0.039831           21.549           0.000
L1.Am

# FHT method

In [86]:
FHT=calculate_FHT(cleansed)

tsmom_model.py:283: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  nonzero=x[x==0].resample(rule='m',how='count')
tsmom_model.py:284: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  Z=(nonzero/x.resample(rule='m',how='count')).dropna()
tsmom_model.py:285: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,center=False).std()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')
tsmom_model.py:285: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')


In [87]:
data=calc_zscore_expanding_window(FHT.dropna(how='all'),3).dropna(how='all')

In [88]:
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

In [89]:
sector_PC={}
sector_variance_explained=pd.DataFrame()

no_pc=3
for sect in d_map.keys():
    try: 
        PPCA=p.PPCA()
        PPCA.fit(np.array(sector_zscores[sect]),d=no_pc)
        sector_variance_explained[sect] = PPCA.var_exp
        pcs=pd.DataFrame()
        for i in range(0,no_pc,1):
            x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
            x.set_axis(sector_zscores[sect].index)
            pcs[i]=x
        sector_PC[sect] = pcs
    except:
        print sect

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  del sys.path[0]


In [90]:
PPCA=p.PPCA()
PPCA.fit(np.array(data),d=no_pc)
sector_variance_explained[sect] = PPCA.var_exp
pcs=pd.DataFrame()
for i in range(0,no_pc,1):
    x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
    x.set_axis(sector_zscores[sect].index)
    pcs[i]=x
sector_PC['Total'] = pcs


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  import sys


In [91]:
prin_factors_FHT=pd.DataFrame()
for sector in sector_PC.keys():
    prin_factors_FHT[sector]=sector_PC[sector].mean(axis=1)

In [92]:
resFHT = VAR.VAR(endog=prin_factors_FHT['2000':'2016'])

In [93]:
resFHT.select_order(14)

                 VAR Order Selection                  
            aic          bic          fpe         hqic
------------------------------------------------------
0       -2.907*      -2.788*     0.05461*      -2.859*
1        -2.742       -1.785      0.06446       -2.354
2        -2.714      -0.9200      0.06640       -1.987
3        -2.831      -0.1989      0.05940       -1.765
4        -2.647       0.8223      0.07207       -1.242
5        -2.712        1.594      0.06859      -0.9676
6        -2.542        2.602      0.08323      -0.4583
7        -2.322        3.659       0.1072       0.1007
8        -2.065        4.753       0.1448       0.6968
9        -1.838        5.818       0.1928        1.264
10       -1.676        6.817       0.2441        1.764
11       -1.448        7.883       0.3368        2.331
12       -2.448        7.720       0.1392        1.671
13       -2.564        8.441       0.1430        1.894
14       -2.789        9.054       0.1360        2.008
* Minimum


{'aic': 0, 'bic': 0, 'fpe': 0, 'hqic': 0}

In [119]:
print resFHT.fit(1).summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 22, May, 2018
Time:                     10:14:23
--------------------------------------------------------------------
No. of Equations:         7.00000    BIC:                   -1.30136
Nobs:                     203.000    HQIC:                  -1.84558
Log likelihood:          -1735.45    FPE:                   0.109147
AIC:                     -2.21535    Det(Omega_mle):       0.0832734
--------------------------------------------------------------------
Results for equation Agriculturals
                      coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------
const                   -0.146670         0.122635           -1.196           0.233
L1.Agriculturals         0.031401         0.116823            0.269           0.788
L1.Currencies            0.038858       